In [1]:
import pandas as pd
#from climtas.nci import GadiClient
import xarray as xr
import datetime as dt
import numpy as np
import dask.array as da

import dask
from dask.distributed import Client
from dask import delayed

In [2]:
client = Client()
client

Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-cpu-bdw-0157.gadi.nci.org.au/41181/proxy/8787/status,
Dashboard: /node/gadi-cpu-bdw-0157.gadi.nci.org.au/41181/proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46675,Workers: 28
Dashboard: /node/gadi-cpu-bdw-0157.gadi.nci.org.au/41181/proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:37635,Total threads: 1
Dashboard: /node/gadi-cpu-bdw-0157.gadi.nci.org.au/41181/proxy/36327/status,Memory: 0 B
Nanny: tcp://127.0.0.1:38003,


In [3]:
#Define lat lon slices (in this case, essentially the whole AUS2200 domain
lon_slice = slice(108,158.98)
lat_slice = slice(-45.7,-6.831799)  

#Define times to slice
t1="2016-01-21 12:00"
t2="2016-01-22 12:00"

#Because boundary layer height data (zmla) is offset by 30 minutes compared with U and V wind data,
# define a separate time slice for later centering
dt_hours=1
t1_zmla = pd.to_datetime(t1) + dt.timedelta(hours=-dt_hours)
t2_zmla = pd.to_datetime(t2) + dt.timedelta(hours=dt_hours)

#We will need to destagger the U and V winds, so we need to load in an extra grid point in lat
# and lon for v and u, respectively. Set up these slices now
dx = 0.022   #Grid spacing in degrees lat-lon
u_lon_slice=slice(lon_slice.start,lon_slice.stop+dx)
v_lat_slice=slice(lat_slice.start,lat_slice.stop+dx)  

In [4]:
def _preprocess(ds):
        ds = ds.sel(lat=lat_slice,lon=lon_slice)
        return ds

In [5]:
def load_aus2200_variable(vnames, t1, t2, exp_id, lon_slice, lat_slice, freq, hgt_slice=None, chunks="auto"):

    '''
    Load static fields for the mjo-enso AUS2200 experiment, stored on the ua8 project

    ## Input

    * vnames: list of names of aus2200 variables

    * t1: start time in %Y-%m-%d %H:%M"

    * t2: start time in %Y-%m-%d %H:%M"
    
    * exp_id: string describing the experiment. either 'mjo-elnino', 'mjo-lanina' or 'mjo-neutral'

    * lat_slice: a slice to restrict lat domain

    * lon_slice: a slice to restrict lon domain

    * freq: time frequency (string). either "10min", "1hr", "1hrPlev"

    * hgt_slice: a slice to restrict data in the vertical (in m)

    * chunks: dict describing the number of chunks. see xr.open_dataset
    '''

    assert exp_id in ['mjo-elnino', 'mjo-lanina', 'mjo-neutral'], "exp_id must either be 'mjo-elnino', 'mjo-lanina' or 'mjo-neutral'"
    assert freq in ["10min", "1hr", "1hrPlev"], "exp_id must either be '10min', '1hr', '1hrPlev'"

    out = []
    for vname in vnames:

        fnames = "/g/data/ua8/AUS2200/"+exp_id+"/v1-0/"+freq+"/"+vname+"/"+vname+"_AUS2200_"+exp_id+"_*.nc"
        ds = xr.open_mfdataset(fnames, chunks=({'lev': -1, 'lat': -1}), 
                               parallel=True, preprocess=_preprocess).sel(time=slice(t1,t2))
        if hgt_slice is not None:
            ds = ds.sel(lev=hgt_slice)
        out.append(ds[vname])

    return out

In [6]:
#Load model level wind data for the sea breeze index
aus2200_ua = load_aus2200_variable(["ua"],t1,t2,"mjo-elnino",u_lon_slice,lat_slice,"1hr",hgt_slice=slice(0,5000),chunks="auto")[0]
aus2200_va = load_aus2200_variable(["va"],t1,t2,"mjo-elnino",lon_slice,v_lat_slice,"1hr",hgt_slice=slice(0,5000),chunks="auto")[0]

In [7]:
aus2200_ua

<xarray.DataArray 'ua' (time: 25, lev: 38, lat: 1963, lon: 2575)> Size: 19GB
dask.array<getitem, shape=(25, 38, 1963, 2575), dtype=float32, chunksize=(1, 38, 1963, 520), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 200B 2016-01-21T12:00:00 ... 2016-01-22T12...
  * lev      (lev) float64 304B 2.5 13.33 33.33 ... 4.433e+03 4.68e+03 4.933e+03
  * lat      (lat) float64 16kB -45.7 -45.68 -45.66 ... -6.891 -6.871 -6.852
  * lon      (lon) float64 21kB 108.0 108.0 108.0 108.1 ... 158.9 159.0 159.0
Attributes:
    standard_name:          eastward_wind
    long_name:              Eastward Wind
    comment:                Zonal wind (positive in a eastward direction).
    units:                  m s-1
    cell_methods:           time: point
    history:                2023-10-21T22:52:27Z altered by CMOR: replaced mi...
    coverage_content_type:  modelResult

In [12]:
lsm

<xarray.DataArray 'lmask' (lat: 1963, lon: 2574)> Size: 40MB
dask.array<mul, shape=(1963, 2574), dtype=int64, chunksize=(1963, 2574), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 16kB -45.7 -45.68 -45.66 ... -6.891 -6.871 -6.852
  * lon      (lon) float64 21kB 108.0 108.0 108.1 108.1 ... 158.9 158.9 159.0

In [9]:
def load_aus2200_static(exp_id,lon_slice,lat_slice,chunks="auto"):

    '''
    Load static fields for the mjo-enso AUS2200 experiment, stored on the ua8 project

    ## Input
    * exp_id: string describing the experiment. either 'mjo-elnino', 'mjo-lanina' or 'mjo-neutral'

    * lat_slice: a slice to restrict lat domain

    * lon_slice: a slice to restrict lon domain
    '''

    assert exp_id in ['mjo-elnino', 'mjo-lanina', 'mjo-neutral'], "exp_id must either be 'mjo-elnino', 'mjo-lanina' or 'mjo-neutral'"
    
    orog = xr.open_mfdataset("/g/data/ua8/AUS2200/"+exp_id+"/v1-0/fx/orog/orog_AUS2200_"+exp_id+"_fx.nc",
                             chunks=chunks, preprocess=_preprocess)
    lsm = xr.open_mfdataset("/g/data/ua8/AUS2200/"+exp_id+"/v1-0/fx/lmask/lmask_AUS2200_"+exp_id+"_fx.nc",
                            chunks=chunks, preprocess=_preprocess)

    return orog.orog, ((lsm.lmask==100)*1)

#Destagger by centering v wind in lat and u wind in lon
lsm = orog, lsm = load_aus2200_static("mjo-elnino",lon_slice,lat_slice)

In [17]:
aus2200_va = (aus2200_va.isel(lat=slice(0,-1)).assign_coords({"lat":lsm.lat}) +
                aus2200_va.isel(lat=slice(1,aus2200_va.lat.shape[0])).assign_coords({"lat":lsm.lat})) / 2
aus2200_ua = (aus2200_ua.isel(lon=slice(0,-1)).assign_coords({"lon":lsm.lon}) +
                aus2200_ua.isel(lon=slice(1,aus2200_ua.lon.shape[0])).assign_coords({"lon":lsm.lon})) / 2
aus2200_wind = xr.Dataset({"u":aus2200_ua, "v":aus2200_va})

ValueError: conflicting sizes for dimension 'lat': length 1962 on <this-array> and length 1963 on {'time': 'time', 'lev': 'lev', 'lon': 'lon', 'lat': 'lat'}

In [22]:
aus2200_va.isel(lat=slice(1,aus2200_va.lat.shape[0])).assign_coords({"lat":lsm.lat})

ValueError: conflicting sizes for dimension 'lat': length 1962 on <this-array> and length 1963 on {'time': 'time', 'lev': 'lev', 'lon': 'lon', 'lat': 'lat'}

In [ ]:
#Load in boundary layer heights
aus2200_zmla = load_aus2200_variable(["zmla"],t1_zmla,t2_zmla,"mjo-elnino",lon_slice,lat_slice,"1hr",chunks="auto")[0]

In [ ]:
#Centre the data in time (offset by one hour compared with u and v wind data)
aus2200_zmla = (aus2200_zmla.isel(time=slice(0,-1)).assign_coords({"time":aus2200_ua.time}) +
                aus2200_zmla.isel(time=slice(1,aus2200_zmla.time.shape[0])).assign_coords({"time":aus2200_ua.time})) / 2

In [ ]:
# Define the function to compute wind direction
def compute_wind_direction(u, v):
    return (90 - np.rad2deg(np.arctan2(-v, -u))) % 360

# Compute `alpha` lazily
alpha = xr.apply_ufunc(
    compute_wind_direction,
    wind_ds["u"].sel({vert_coord: alpha_height}, method="nearest"),
    wind_ds["v"].sel({vert_coord: alpha_height}, method="nearest"),
    dask="parallelized",  # Ensures Dask compatibility
    output_dtypes=[float],  # Specify output dtype
)

# Compute `beta` lazily
beta = xr.apply_ufunc(
    compute_wind_direction,
    wind_ds["u"],
    wind_ds["v"],
    dask="parallelized",  # Ensures Dask compatibility
    output_dtypes=[float],  # Specify output dtype
)

# Add the computed variables back to the dataset
wind_ds = wind_ds.assign(alpha=alpha, beta=beta)

In [ ]:
# Define the function to compute sbi
def compute_sbi(alpha, beta, theta):
    return (
        np.cos(np.deg2rad(alpha - theta)) *
        np.cos(np.deg2rad(alpha + 180 - beta))
    )

# Compute `sbi` lazily
sbi = xr.apply_ufunc(
    compute_sbi,
    wind_ds["alpha"],  # Alpha from the dataset
    wind_ds["beta"],   # Beta from the dataset
    theta,             # Theta, could be a scalar or DataArray
    dask="parallelized",  # Enable Dask compatibility
    output_dtypes=[float],  # Specify output dtype
)

# Add the computed `sbi` back to the dataset
wind_ds = wind_ds.assign(sbi=sbi)

In [ ]:
def calc_sbi(wind_ds,
                    angle_ds,
                    subtract_mean=False,
                    height_mean=True,
                    blh_da=None,
                    alpha_height=0,
                    sb_heights=[500,2000],
                    mean_heights=[0,4500],
                    height_method="blh",
                    blh_rolling=0,
                    vert_coord="height"):

    '''
    Take a xarray dataset of u and v winds, as well as a dataset of coastline angles, and apply the algorithm of Hallgren et al. (2023) to compute the sea breeze index via a single-column method.

    ### Input
    * wind_ds: An xarray dataset with "u" and "v" wind component variables, and a vertical coordinate "height" in metres

    * angle_ds: An xarray dataset of coastline orientation angles (degrees from N) 

    * subtract_mean: Boolean option for whether to subtract the mean background wind, and calculate perturbation sbi and lbi. Currently using an arithmatic mean over mean_heights layer

    * height_mean: Boolean option to control subtract_mean method. If true, subtract the height mean over mean_heights. Otherwise, subtract the daily mean

    * mean_heights: Array of size (2) that describes the bounds used to mean wind layer. Used if subtract_mean=True

    * blh_da: An xarray dataarray with boundary layer heights, also in m. Used if height_method="blh" to define heights to look for sea/land breezes.

    * alpha_height: Height level in m to define the "low-level" wind

    * sb_heights: Array of size (2) that describes the bounds used to define the upper level sea breeze height. Used if height_method="static"

    * lb_heights: Array of size (2) that describes the bounds used to define the upper level land breeze height. Used if height_method="static"

    * height_method: String used to choose the method for selecting upper level heights to define a circulation. Either "static" or "blh". "static" uses static height limits defined by lb_heights/sb_heights, blh uses the blh_ds.

    * blh_rolling: Integer used to define the number of rolling time windows over which to take the maximum. If zero then no rolling max is taken.

    ### Output
    * xarray dataset with sbi
    '''

    #Subtract the mean wind. Define mean as the mean over mean_heights m level, or the daily mean
    if subtract_mean:
        if height_mean:
            u_mean, v_mean = vert_mean_wind(wind_ds,mean_heights,vert_coord)
        else:
            u_mean, v_mean = daily_mean_wind(wind_ds)
        wind_ds["u"] = wind_ds["u"] - u_mean
        wind_ds["v"] = wind_ds["v"] - v_mean

    #Convert coastline orientation angle to the angle perpendicular to the coastline (pointing away from coast. from north)
    theta = (((angle_ds.angle_interp+180)%360-90)%360)

    #Calculate wind directions (from N) for low level (alpha) and all levels (beta)
    alpha = (90 - np.rad2deg(np.arctan2(
        -wind_ds["v"].sel({vert_coord:alpha_height},method="nearest"),
        -wind_ds["u"].sel({vert_coord:alpha_height},method="nearest")))) % 360
    beta = (90 - np.rad2deg(np.arctan2(
        -wind_ds["v"], 
        -wind_ds["u"]))) % 360
    wind_ds["alpha"] = alpha
    wind_ds["beta"] = beta

    #Calculate the sea breeze and land breeze indices
    sbi = np.cos(np.deg2rad((wind_ds.alpha - theta))) * \
            np.cos(np.deg2rad(wind_ds.alpha + 180 - wind_ds.beta))

    #Mask to zero everywhere except for the following conditions
    sb_cond = ( (np.cos(np.deg2rad((wind_ds.alpha - theta)))>0), #Low level flow onshore
            (np.cos(np.deg2rad(wind_ds.beta - (theta+180)))>0), #Upper level flow offshore
            (np.cos(np.deg2rad(wind_ds.alpha + 180 - wind_ds.beta))>0) #Upper level flow opposing
                  )
    sbi = xr.where(sb_cond[0] & sb_cond[1] & sb_cond[2], sbi, 0)

    #Return the max over some height. Either defined statically or boundary layer height
    _,_,_,hh = da.meshgrid(
        da.rechunk(da.array(wind_ds[wind_ds.u.dims[0]]),chunks={0:wind_ds.u.chunksizes[wind_ds.u.dims[0]][0]}),
        da.rechunk(da.array(wind_ds[wind_ds.u.dims[1]]),chunks={0:wind_ds.u.chunksizes[wind_ds.u.dims[1]][0]}),
        da.rechunk(da.array(wind_ds[wind_ds.u.dims[2]]),chunks={0:wind_ds.u.chunksizes[wind_ds.u.dims[2]][0]}),
        da.rechunk(da.array(wind_ds[wind_ds.u.dims[3]]),chunks={0:wind_ds.u.chunksizes[wind_ds.u.dims[3]][0]}), indexing="ij")
    wind_ds["height_var"] = (wind_ds.u.dims,hh)
    if height_method=="static":
        sbi = xr.where((sbi[vert_coord] >= sb_heights[0]) & (sbi[vert_coord] <= sb_heights[1]),sbi,0)
    elif height_method=="blh":
        if blh_rolling > 0:
            blh_da = blh_da.rolling({"time":blh_rolling}).max()
        sbi = xr.where((wind_ds.height_var <= blh_da),sbi,0)
    else:
        raise ValueError("Invalid height method")

    #Compute each index as the max in the column
    sbi = sbi.max(vert_coord)
    sbi_ds = xr.Dataset({"sbi":sbi})        

    return sbi_ds

def vert_mean_wind(wind_ds,mean_heights,vert_coord):

    """
    For an xarray dataset with u and v winds, take the vertical mean over some layer.
    """

    u_mean = wind_ds["u"].sel({vert_coord:slice(mean_heights[0],mean_heights[1])}).mean(vert_coord)
    v_mean = wind_ds["v"].sel({vert_coord:slice(mean_heights[0],mean_heights[1])}).mean(vert_coord)

    return u_mean, v_mean

def daily_mean_wind(wind_ds):

    """
    For an xarray dataset with u and v winds, take a rolling daily mean
    """

    dt_h = np.round((wind_ds.time.diff("time")[0].values / (1e9 * 60 * 60)).astype(float)).astype(int)
    time_window = int(24 / dt_h)
    min_periods = int(time_window/2)

    u_mean = wind_ds["u"].rolling(dim={"time":time_window},center=True,min_periods=min_periods).mean()
    v_mean = wind_ds["v"].rolling(dim={"time":time_window},center=True,min_periods=min_periods).mean()

    return u_mean, v_mean
    

#Need to first load in a dataset of coastline angles
#angle_ds = xr.open_dataset("/scratch/public/ab4502/aus2200_v3.nc").sel(lat=lat_slice,lon=lon_slice)
angle_ds = xr.open_dataset("/g/data/gb02/ab4502/coastline_data/aus2200_v3.nc").sel(lat=lat_slice,lon=lon_slice)

#Call function
sbi = calc_sbi(aus2200_wind,
        angle_ds,
        subtract_mean=False,
        height_mean=False,
        blh_da=aus2200_zmla,
        alpha_height=0,
        sb_heights=[500,2000],
        mean_heights=[0,4500],
        height_method="blh",
        blh_rolling=0,
        vert_coord="lev")
sbi.sbi.sel(time="2016-01-22 08:00").plot()

In [ ]:
#Call function again, but this time subtract the daily mean u and v wind components.
#Before calling the function, rechunk to one time chunk for efficiency
sbi = calc_sbi(aus2200_wind.chunk({"time":-1,"lat":100,"lon":100}),
        angle_ds,
        subtract_mean=True,
        height_mean=False,
        blh_da=aus2200_zmla.chunk({"time":-1,"lat":100,"lon":100}),
        alpha_height=0,
        sb_heights=[500,2000],
        mean_heights=[0,4500],
        height_method="blh",
        blh_rolling=0,
        vert_coord="lev")
sbi.sbi.sel(time="2016-01-22 08:00").plot()

In [ ]:

time_window = 24
min_periods = 12
u_mean = aus2200_wind["u"].\
            chunk({"time":-1,"lat":100,"lon":100}).\
            rolling(dim={"time":time_window},center=True,min_periods=min_periods).\
            mean()
#u_mean.to_netcdf("/scratch/gb02/ab4502/tmp/u_mean.nc")

In [27]:
time_window = 24
min_periods = 12
u_mean = aus2200_ua.rolling(dim={"time":time_window},center=True,min_periods=min_periods).mean()

In [33]:
u_mean_c = u_mean.compute()

2024-12-02 15:49:36,280 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:45023 -> tcp://127.0.0.1:38251
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
TimeoutError: [Errno 110] Connection timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/worker.py", line 1797, in get_data
    response = await comm.read(deserializers=serializers)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/comm/

In [30]:
u_mean

<xarray.DataArray 'ua' (time: 25, lev: 38, lat: 1963, lon: 2575)> Size: 19GB
dask.array<truediv, shape=(25, 38, 1963, 2575), dtype=float32, chunksize=(24, 10, 491, 130), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 200B 2016-01-21T12:00:00 ... 2016-01-22T12...
  * lev      (lev) float64 304B 2.5 13.33 33.33 ... 4.433e+03 4.68e+03 4.933e+03
  * lat      (lat) float64 16kB -45.7 -45.68 -45.66 ... -6.891 -6.871 -6.852
  * lon      (lon) float64 21kB 108.0 108.0 108.0 108.1 ... 158.9 159.0 159.0
Attributes:
    standard_name:          eastward_wind
    long_name:              Eastward Wind
    comment:                Zonal wind (positive in a eastward direction).
    units:                  m s-1
    cell_methods:           time: point
    history:                2023-10-21T22:52:27Z altered by CMOR: replaced mi...
    coverage_content_type:  modelResult